In [ ]:
library(ggplot2)
library(dplyr)
library(lme4)
library(lmerTest)
library(glmmTMB)
library(cowplot)
library(grid)
library(car)

pallete = c("#318480", "#b4464f")

ignore <- c("augment_tied") # models not to plot


# Bock 1992

In [ ]:
b92_wiki_df <- read.csv("../evalsets/Bock1992/items.csv.reformat")
b92_ptb_df <- read.csv("../evalsets/Bock1992/items.ptb.csv.reformat")


b92_df <- rbind(b92_ptb_df, b92_wiki_df)
b92_df <- tidyr::separate(b92_df, condition, c("nums","clause"), sep="_")
b92_df <- tidyr::separate(b92_df, nums, c("subj", "attr"), sep=1:2)

b92_df

b92_df$type <- substr(b92_df$model, 5, nchar(b92_df$model)-1)
b92_df[b92_df$model == "gpt2",]$type <- "gpt2"
b92_df$type <- as.factor(b92_df$type)
b92_df$match <- as.factor(b92_df$subj == b92_df$attr)
b92_df$subj <- as.factor(b92_df$subj)
b92_df$clause <- as.factor(b92_df$clause)

b92_df$model <- as.factor(b92_df$model)

coding_matrix <- matrix(c(1, -1), ncol=1)
contrasts(b92_df$match) <- coding_matrix
contrasts(b92_df$clause) <- coding_matrix
contrasts(b92_df$clause)
b92_df$sample <- 100 - b92_df$sample
b92_df$max <- 100 - b92_df$max


summary(b92_df)

In [ ]:

means <- b92_df %>% group_by(type, match, clause) %>% summarize(sample = mean(sample), max = mean(max))
means_m <- b92_df %>% group_by(model, type, match, clause) %>% summarize(sample = mean(sample), max = mean(max))

means_na <- b92_df %>% group_by(type, match, subj) %>% summarize(sample = mean(sample), max = mean(max))

means <- subset(means, !(type %in% ignore))
means_m <- subset(means_m, !(type %in% ignore))

means_na

means_na_m <- b92_df %>% group_by(model, type, match, subj) %>% summarize(sample = mean(sample), max = mean(max))


In [ ]:
options(repr.plot.width=4, repr.plot.height=4)


b92_human_ <- data.frame(type = rep(c("h"), 8),
                        clause=c("prop", "prop", "prop", "prop", "rc", "rc", "rc","rc"),
                        match=c(FALSE, FALSE, TRUE, TRUE, TRUE, TRUE, FALSE, FALSE),
                        attr=c("S","P", "S", "P", "S", "P", "S","P"),
                        subj=c("p", "s", "s", "p", "s", "p", "p", "s"),
                        sample=c(8/188, 29/169, 3/187,6/180, 1/209, 3/199, 3/180, 22/189)
                       )
b92_human <- b92_human_ %>% group_by(type, match, clause) %>% 
                           summarize(sample = mean(sample))


b92_human$type <- as.factor(b92_human$type)
b92_human$match <- as.factor(b92_human$match)
b92_human$clause <- as.factor(b92_human$clause)
b92_human$sample <- b92_human$sample * 100
b92_human$max <- b92_human$sample
b92_human

means_human <- rbind(means, b92_human)
means_human <- subset(means_human, !(type %in% ignore))
means_human
means_human_na <- b92_human_ %>% group_by(type, match, subj) %>% 
                  summarize(sample = mean(sample))
means_human_na$type <- as.factor(means_human_na$type)
means_human_na$match <- as.factor(means_human_na$match)
means_human_na$subj <- as.factor(means_human_na$subj)
means_human_na$sample <- means_human_na$sample * 100
means_human_na$max <- means_human_na$sample
means_human_na <-rbind(means_na, means_human_na)
means_human_na

In [ ]:
options(repr.plot.width=6.5, repr.plot.height=4)
model_order = c("lmaug", "augment", "wiki_lm", "augment_tied", "ccglm", "gpt2", "h")

b92_df$type <- factor(b92_df$type, levels=model_order)
b92_df <- subset(b92_df, !(type %in% ignore))

ggplot(data=b92_df, aes(x=factor(type, levels=model_order), color=clause, linetype=clause, y=sample/100)) +
geom_point(data=subset(means_human, match==TRUE), position=position_dodge(width=0.5), aes(shape="m"), size=2, fill="#FFFFFF") + 
# need ^ for the x axis order? don't ask me why tbh
stat_summary(data=subset(b92_df, match==TRUE), geom="errorbar", width=0.5, 
             position=position_dodge(width=0.5), aes(x=factor(type, levels=model_order), group=clause)) +
stat_summary(data=subset(b92_df, match==FALSE), geom="errorbar", width=0.5, 
             position=position_dodge(width=0.5), aes(x=factor(type, levels=model_order), group=clause)) +
geom_line(data=means_human, position=position_dodge(width=0.5), size=1.2) +
geom_point(data=subset(means_human, match==TRUE), 
           position=position_dodge(width=0.5), aes(shape="m"), size=3, fill="#FFFFFF") +
geom_point(data=subset(means_human, match==FALSE), 
           position=position_dodge(width=0.5), aes(shape="mm"), size=3, fill="#FFFFFF") + 
labs(x="", 
     color="", shape="", y="% Attraction Errors", title="", linetype="") +
scale_y_continuous(labels=scales::percent) +
scale_color_manual(labels=c("prop"="PP Attractor", "rc"="RC Attractor"), values=pallete) +
scale_linetype_manual(labels=c("prop"="PP Attractor", "rc"="RC Attractor"), values=c("prop"="1111", "rc"="solid")) +
scale_x_discrete(labels=c("lmaug"="LM Only \nWiki+PTB",
                          "augment"="LM+CCG \nWiki+PTB",
                          "wiki_lm"="LM Only \nWiki",
                          "augment_tied"="LM+CCG \nWIki+PTB\nTied",
                          "ccglm"="LM Only \nPTB",
                          "gpt2"="GPT-2",
                          "h"="Humans")) +
scale_shape_manual(labels=c("m"="Match",
                            "mm"="Mismatch"),
                   values=c("m"=21,
                            "mm"=22)) +
coord_cartesian(ylim = c(0.0,1.0)) +
theme_bw() +
theme(axis.text.x=element_text(size=12, angle=45, hjust=1), axis.text.y=element_text(size=12),
      axis.title.y=element_text(size=12), axis.title.x=element_text(size=10),
      strip.text=element_text(size=12), legend.text=element_text(size=12))


ggsave("plots/Bock_all.png", width= 6.5, height = 4)

In [ ]:
b92_lm <- glmmTMB(data=subset(b92_df, type=="lmaug"), 
                  sample/100 ~ subj * match * clause + (1 | item) + (1 | model),
                 family=beta_family())
summary(b92_lm)

b92_a <- glmmTMB(data=subset(b92_df, type=="augment"), 
                  sample/100 ~ subj * match * clause + (1 | item) + (1 | model),
                 family=beta_family())
summary(b92_a)


b92_la <- glmmTMB(data=subset(b92_df, type=="lmaug" | type=="augment"), 
                  sample/100 ~ subj * match * clause * type + (1 | item) + (1 | model),
                 family=beta_family())

summary(b92_la)


b92_c <- glmmTMB(data=subset(b92_df, type=="ccglm"), 
                  sample/100 ~ subj * match * clause + (1 | item) + (1 | model),
                 family=beta_family())
summary(b92_c)


b92_w <- glmmTMB(data=subset(b92_df, type=="wiki_lm"), 
                  sample/100 ~ subj * match * clause + (1 | item) + (1 | model),
                 family=beta_family())
summary(b92_w)

b92_cw <- glmmTMB(data=subset(b92_df, type=="ccglm" | type=="wiki_lm"), 
                  sample/100 ~ subj * match * clause * type + (1 | item) + (1 | model),
                 family=beta_family())

summary(b92_cw)

b92_g <- glmmTMB(data=subset(b92_df, type=="gpt2"), 
                  sample/100 ~ subj * match * clause + (1 | item) + (1 | model),
                 family=beta_family())
summary(b92_g)

In [ ]:
pval_string <- function(p) {
    if (p < 0.001) {
        return ("p < 0.001 ***")
    } else if (p < 0.01) {
        return ("p < 0.01 *")
    } else if (p < 0.05) {
        return ("p < 0.05 *")
    }
    else {return(paste0("p = ", round(p,2)))}
}

stats_strings <- function(model) {
    x <- coef(summary(b92_a))$cond
    for(i in rownames(x)) {
        y <- x[i,]
        print(paste0(i, ": ",
                    "\beta = ",round(y[["Estimate"]], 2), 
                    ", |z| = ", round(y[["z value"]], 2), 
                    ", ", pval_string(y[["Pr(>|z|)"]])
        ))
    }
}

In [ ]:
stats_strings(ma)

# Haskell & Macdonald 2011

In [ ]:
hm_wiki_df <- read.csv("../evalsets/HaskellMacdonald2011/items.csv.reformat")
hm_ptb_df <- read.csv("../evalsets/HaskellMacdonald2011/items.ptb.csv.reformat")


hm_df <- rbind(hm_wiki_df, hm_ptb_df)

hm_df$type <- substr(hm_df$model,5, nchar(hm_df$model) -1)
hm_df[hm_df$model == "gpt2",]$type <- "gpt2"
hm_df$type <- as.factor(hm_df$type)
hm_df$condition <- as.factor(hm_df$condition)

contrasts(hm_df$condition)

coding_matrix <- matrix(c(1, -1), ncol=1)
contrasts(hm_df$condition) <- coding_matrix



hm_df

In [ ]:
output <- hm_df %>% group_by("type"=type, "order"=condition) %>% summarize("sample"=mean(sample), "max" = mean(max))
hm_means_m <- hm_df %>% group_by("model"=model, "type"=type, "order"=condition) %>% summarize("sample"=mean(sample), "max" = mean(max))

hm_means_m <- subset(hm_means_m, !(type %in% ignore))

hm_human <- data.frame(type=c("h", "h"),
                       order=c("SP", "PS"),
                       sample=c(98, 28))

hm_human$max = hm_human$sample

hm_means <- rbind(output, hm_human)
hm_means <- subset(hm_means, !(hm_means$type %in% ignore))

output

In [ ]:
options(repr.plot.width=6.5, repr.plot.height=4)
pallette2 = c("#052F5F", "#DA9307")

hm_df <- subset(hm_df, !(type %in% ignore))
model_order = c("lmaug", "augment", "wiki_lm", "ccglm", "gpt2", "h")
hm_df$type <- factor(hm_df$type, levels=model_order)
hm_means$type <- factor(hm_means$type, levels=model_order)
levels(hm_means$type)

ggplot(data=hm_means, aes(x=type, y=sample/100, color=order)) +
geom_point(data=subset(hm_means, order=="SP"), aes(shape="m", color=pallette2[[1]]), size=3) +
geom_point(data=subset(hm_means, order=="PS"), aes(shape="mm", color=pallette2[[2]]), size=3) + 
stat_summary(data=subset(hm_df, condition=="SP"), geom="errorbar", width=0.5, 
                         aes(x=type, group=condition), color=pallette2[[1]]) +
stat_summary(data=subset(hm_df, condition=="PS"), geom="errorbar", width=0.5,  
                         aes(x=type, group=condition), color=pallette2[[2]]) +
geom_line(data=hm_means, position=position_dodge(width=0.5), size=1.2, color="#999999") +
geom_point(data=subset(hm_means, order=="SP"), aes(shape="m", color=pallette2[[1]]), size=3, fill="#FFFFFF") +
geom_point(data=subset(hm_means, order=="PS"), aes(shape="mm", color=pallette2[[2]]), size=3, fill="#FFFFFF") + 
labs(x="", 
     color="", shape="", y="% Plural Agreement", title="") +
scale_y_continuous(labels=scales::percent) +
scale_x_discrete(labels=c("lmaug"="LM Only \nWiki+PTB",
                          "augment"="LM+CCG \nWiki+PTB",
                          "wiki_lm"="LM Only \nWiki",
                          "ccglm"="LM Only \nPTB",
                          "gpt2"="GPT-2",
                          "h"="Humans")) +
scale_shape_manual(labels=c("The boy or the girls",
                            "The boys or the girl"),
                   values=c(21,
                            22)) +
scale_color_manual(name="",
                   labels=c("The boy or the girls",
                            "The boys or the girl"),
                     values=c(pallette2[[1]],
                             pallette2[[2]])) +
coord_cartesian(ylim = c(0,1.0)) +
theme_bw() +
theme(axis.text.x=element_text(size=12, angle=45, hjust=1), axis.text.y=element_text(size=12),
      axis.title.y=element_text(size=12), axis.title.x=element_text(size=10),
      strip.text=element_text(size=12), legend.text=element_text(size=12), legend.position="top")


ggsave("plots/Haskell_all.png", width= 6.5, height = 4)

In [ ]:
hm_lm <- glmmTMB(data=subset(hm_df, hm_df$type == "lmaug"), 
                 sample/100 ~ condition  + (1|item) + (1 | model),
                family=beta_family())

summary(hm_lm)

hm_lm <- glmmTMB(data=subset(hm_df, hm_df$type == "augment"), 
                 sample/100 ~ condition  + (1|item) + (1 | model),
                family=beta_family())

summary(hm_lm)

hm_lm <- glmmTMB(data=hm_df, 
                 sample/100 ~ condition * type + (1|item) + (1 | model),
                family=beta_family())

summary(hm_lm)
hm_lm <- glmmTMB(data=subset(hm_df, type == "wiki_lm" | type == "ccglm"), 
                 sample/100 ~ condition * type + (1|item) + (1 | model),
                family=beta_family())

summary(hm_lm)

hm_lm <- glmmTMB(data=subset(hm_df, type == "gpt2"), 
                 sample/100 ~ condition + (1|item) + (1 | model),
                family=beta_family())

summary(hm_lm)

# Franck 2002

In [ ]:
franck_wiki_df <- read.csv('../evalsets/Franck2002/items.csv.reformat')
franck_ptb_df <- read.csv('../evalsets/Franck2002/items.ptb.csv.reformat')

franck_df = rbind(franck_wiki_df, franck_ptb_df)

franck_df <- tidyr::separate(franck_df, condition, c("subj", "inter", "local"), sep=1:3)
franck_df$sample <- 100 - franck_df$sample
franck_df$max <- 100 - franck_df$max

franck_df$inter_match <- as.factor(franck_df$subj == franck_df$inter)
franck_df$local_match <- as.factor(franck_df$subj == franck_df$local)
franck_df$type = substr(franck_df$model, 5, nchar(franck_df$model) -1)
franck_df[franck_df$model == "gpt2",]$type <- "gpt2"
franck_df$type <- as.factor(franck_df$type)

contrasts(franck_df$local_match) <- coding_matrix
contrasts(franck_df$inter_match) <- coding_matrix

franck_df
summary(franck_df)

In [ ]:
franck_means <- franck_df %>% group_by(type, inter_match, local_match) %>% 
                       summarize(sample = mean(sample), max = mean(max))
franck_means_m <- franck_df %>% group_by(model, type, inter_match, local_match) %>% 
                       summarize(sample = mean(sample), max = mean(max))

franck_human <- data.frame(type=rep(c("h"), 8),
                           subj=rep(c("S","P"), each=4),
                           inter=rep(c("S", "S", "P", "P"), times=2),
                           local=rep(c("S", "P"), times=4),
                           sample=c(1.3, 0, 9.8, 4.5, 5.8, 9.4, 3.1, 1.3))

franck_human$inter_match <- as.factor(franck_human$inter == franck_human$subj)
franck_human$local_match <- as.factor(franck_human$local == franck_human$subj)

franck_human$max <- franck_human$sample

franck_human <- franck_human %>% group_by(type, inter_match, local_match) %>% 
                       summarize(sample = mean(sample), max = mean(max))


franck_means <- rbind(franck_means, franck_human)

franck_means <- subset(franck_means, !(franck_means$type %in% ignore))
franck_means_m <- subset(franck_means_m, !(franck_means_m$type %in% ignore))
franck_means

In [ ]:
library(repr)
options(repr.plot.width=6.5, repr.plot.height=3.25)

model_order = c("lmaug", "augment", "wiki_lm", "ccglm", "gpt2", "h")

plt_1 <- ggplot(data=franck_means, aes(x=local_match, color=inter_match, group=local_match, y=sample/100)) +
facet_grid(~factor(type, levels=model_order), labeller=as_labeller(c("lmaug"="LM Only \nWiki+PTB",
                          "augment"="LM+CCG \nWiki+PTB",
                          "wiki_lm"="LM Only \nWiki",
                          "ccglm"="LM Only \nPTB",
                          "gpt2"="GPT-2",
                          "h"="Humans")))+
geom_point(data=subset(franck_means, inter_match==FALSE),
           shape=16, size=3) + 
geom_point(data=subset(franck_means, inter_match==TRUE),
           shape=18, size=3) + 
stat_summary(data=subset(franck_means_m, inter_match==TRUE), geom="errorbar", width=0.5, position=position_dodge(width=0.7)) +
stat_summary(data=subset(franck_means_m, inter_match==FALSE), geom="errorbar", width=0.5, position=position_dodge(width=0.7)) +
geom_line(data=franck_means, 
          size=1) +
geom_line(data=franck_means,
          aes(color=inter_match, group=inter_match), 
          linetype="dashed", size=0.7) +
labs(x="Linearly Closer Attractor (Company(s))", 
     color="Syntactically Closer Attractor \nPresident(s)", y="% Attraction Errors", title="") +
scale_y_continuous(labels=scales::percent) +
scale_color_manual(labels=c("Mismatch", 
                        "Match"), values=pallete) +
scale_shape_manual(labels=c("LM+CCG", 
                        "LM Only"), values=pallete) +
scale_x_discrete(labels=c("FALSE"="Mismatch", "TRUE"="Match")) +
coord_cartesian(ylim = c(0,0.8)) +
theme_bw() +
theme(axis.text.x=element_text(size=12, angle=45, hjust=1), axis.text.y=element_text(size=12),
      axis.title.y=element_text(size=12), axis.title.x=element_text(size=12),
      strip.text=element_text(size=10), legend.text=element_text(size=12), 
      legend.title=element_text(size=12), strip.background=element_rect(fill="white", linetype=0), 
      strip.placement="outside")

plt_1
ggsave("plots/Franck_all.png", width=6.5, height=3.25)

In [ ]:
franck_inter <- franck_means %>% 
                group_by(type, inter_match) %>% 
                summarize(sample = mean(sample), max = mean(max), position="inter", match=inter_match)
franck_local <- franck_means %>% 
                group_by(type, local_match) %>% 
                summarize(sample = mean(sample), max = mean(max), position="local", match=local_match)

franck_bypos <- rbind(franck_inter, franck_local)


franck_bypos$position <- as.factor(franck_bypos$position)

franck_bypos$match <- as.logical(franck_bypos$match)

franck_inter_m <- franck_means_m %>% 
                group_by(model, type, inter_match) %>% 
                summarize(sample = mean(sample), max = mean(max), position="inter", match=inter_match)
franck_local_m <- franck_means_m %>% 
                group_by(model, type, local_match) %>% 
                summarize(sample = mean(sample), max = mean(max), position="local", match=local_match)

franck_bypos_m <- rbind(franck_inter_m, franck_local_m)


franck_bypos_m$position <- as.factor(franck_bypos_m$position)

franck_bypos_m$match <- as.logical(franck_bypos_m$match)


In [ ]:
library(repr)
options(repr.plot.width=6.5, repr.plot.height=4)

pallete3 = c("#318480", "#b4464f")

model_order = c("lmaug", "augment", "wiki_lm", "ccglm", "gpt2", "h")

plt_1 <- ggplot(data=franck_bypos, aes(x=position, color=position, y=sample/100)) +
facet_grid(~factor(type, levels=model_order), labeller=as_labeller(c("lmaug"="LM Only \nWiki+PTB",
                          "augment"="LM+CCG \nWiki+PTB",
                          "wiki_lm"="LM Only \nWiki",
                          "ccglm"="LM Only \nPTB",
                          "gpt2"="GPT-2",
                          "h"="Humans")))+
geom_point(data=subset(franck_bypos, match==FALSE),
           shape=21, size=0, fill="#FFFFFF") + 
stat_summary(data=subset(franck_bypos_m, match==TRUE), geom="errorbar", width=0.5, 
             position=position_dodge(width=0.7)) +
stat_summary(data=subset(franck_bypos_m, match==FALSE), geom="errorbar", width=0.5, 
             position=position_dodge(width=0.7)) +
geom_line(data=franck_bypos,
          aes(color=position), 
          linetype="dashed", size=1) +
geom_point(data=subset(franck_bypos, match==FALSE),
           shape=21, size=3, fill="#FFFFFF") + 
geom_point(data=subset(franck_bypos, match==TRUE),
           shape=22, size=3, fill="#FFFFFF") + 
labs(x="", 
     color="", y="% Attraction Errors", title="") +
scale_y_continuous(labels=scales::percent) +
scale_color_manual(labels=c("FALSE"="Mismatch", 
                        "TRUE"="Match"), values=pallete) +
scale_x_discrete(labels=c("inter"="Syn. Closer", "local"="Lin. Closer")) +
coord_cartesian(ylim = c(0,0.8)) +
theme_bw() +
theme(axis.text.x=element_text(size=12, angle=45, hjust=1), axis.text.y=element_text(size=12),
      axis.title.y=element_text(size=12), axis.title.x=element_text(size=10),
      strip.text=element_text(size=12), legend.text=element_text(size=12), 
      legend.title=element_text(size=12), strip.background=element_rect(fill="white", linetype=0), 
      strip.placement="outside", legend.position="none")

plt_1
ggsave("plots/Franck_eff_all.png", width=6.5, height=4)

In [ ]:
library(repr)
options(repr.plot.width=6.5, repr.plot.height=4)

pallete3 = c("#318480", "#b4464f")

model_order = c("h", "lmaug", "augment")

plt_1 <- ggplot(data=subset(franck_bypos, type %in% model_order), aes(x=position, color=position, y=sample/100)) +
facet_grid(~factor(type, levels=model_order), labeller=as_labeller(c("lmaug"="LM Only",
                          "augment"="LM+CCG",
                          "wiki_lm"="LM Only \nWiki",
                          "ccglm"="LM Only \nPTB",
                          "gpt2"="GPT-2",
                          "h"="Humans")))+
geom_point(data=subset(franck_bypos, (match==FALSE) &(type %in% model_order)),
           shape=21, size=0, fill="#FFFFFF") + 
stat_summary(data=subset(franck_bypos_m, (match==TRUE) &(type %in% model_order)), geom="errorbar", width=0.35, 
             position=position_dodge(width=0.7)) +
stat_summary(data=subset(franck_bypos_m, (match==FALSE) &(type %in% model_order)), geom="errorbar", width=0.35, 
             position=position_dodge(width=0.7)) +
geom_line(data=subset(franck_bypos,type %in% model_order),
          aes(color=position), 
          linetype="dashed", size=1) +
geom_point(data=subset(franck_bypos, (match==FALSE) &(type %in% model_order)),
           shape=21, size=2, fill="#FFFFFF") + 
geom_point(data=subset(franck_bypos, (match==TRUE) &(type %in% model_order)),
           shape=22, size=2, fill="#FFFFFF") + 
labs(x="", 
     color="", y="% Attraction Errors", title="") +
scale_y_continuous(labels=scales::percent) +
scale_color_manual(labels=c("FALSE"="Mismatch", 
                        "TRUE"="Match"), values=pallete) +
scale_x_discrete(labels=c("inter"="Syn. Closer", "local"="Lin. Closer")) +
coord_cartesian(ylim = c(0,0.8)) +
theme_bw() +
theme(axis.text.x=element_text(size=12, angle=45, hjust=1), axis.text.y=element_text(size=12),
      axis.title.y=element_text(size=12), axis.title.x=element_text(size=10),
      strip.text=element_text(size=12), legend.text=element_text(size=12), 
      legend.title=element_text(size=12), strip.background=element_rect(fill="white", linetype=0), 
      strip.placement="outside", legend.position="none")

plt_1
ggsave("plots/Franck_eff_lmccglmonly.png", width=4.5, height=4)

In [ ]:
#LM-Only
franck_lm <- glmmTMB(sample/100 ~ inter_match * local_match + (1 | item) 
                  + (1 | model), subset(franck_df, type=="lmaug"), 
                   family=beta_family())

summary(franck_lm)

# Hack together a linear hypothesis test for a glmmTMB model with hypothesis inter_match = local_match
hypothesis= c(0,1,-1,0)
linearHypothesis(franck_lm, hypothesis, 
                 vcov. =vcov(franck_lm)[["cond"]], 
                 coef. =fixef(franck_lm)[["cond"]])

In [ ]:
# LM+CCG
franck_aug <- glmmTMB(sample/100 ~ inter_match * local_match + (1 | item) 
                  + (1 | model), subset(franck_df, type=="augment"), 
                   family=beta_family())
summary(franck_aug)

hypothesis= c(0,1,-1,0)
linearHypothesis(franck_aug, hypothesis, 
                 vcov. =vcov(franck_aug)[["cond"]], 
                 coef. =fixef(franck_aug)[["cond"]])


In [ ]:
franck_la_df <- droplevels(subset(franck_df, type=="a" | type=="l"))
summary(franck_la_df)
franck_both <- glmmTMB(sample/100 ~ inter_match * local_match*type + (1 | item) 
                  + (1 | model), data=franck_la_df, 
                   family=beta_family())
summary(franck_both)

In [ ]:
# LM PTB
franck_ptb <- glmmTMB(sample/100 ~ inter_match * local_match + (1 | item) 
                  + (1 | model), subset(franck_df, type=="c"), 
                   family=beta_family())
summary(franck_ptb)
hypothesis= c(0,1,-1,0)
linearHypothesis(franck_ptb, hypothesis, 
                 vcov. =vcov(franck_ptb)[["cond"]], 
                 coef. =fixef(franck_ptb)[["cond"]])


In [ ]:
# LM Wiki
franck_wiki <- glmmTMB(sample/100 ~ inter_match * local_match + (1 | item) 
                  + (1 | model), subset(franck_df, type=="w"), 
                   family=beta_family())
summary(franck_wiki)
hypothesis= c(0,1,-1,0)
linearHypothesis(franck_wiki, hypothesis, 
                 vcov. =vcov(franck_wiki)[["cond"]], 
                 coef. =fixef(franck_wiki)[["cond"]])


In [ ]:
franck_la_df <- droplevels(subset(franck_df, type=="c" | type=="w"))
summary(franck_la_df)
franck_both <- glmmTMB(sample/100 ~ inter_match * local_match*type + (1 | item) 
                  + (1 | model), data=franck_la_df, 
                   family=beta_family())
summary(franck_both)

In [ ]:
# gpt2
franck_gpt2 <- glmmTMB(sample/100 ~ inter_match * local_match + (1 | item) 
                  + (1 | model), subset(franck_df, type=="gpt2"), 
                   family=beta_family())
summary(franck_gpt2)
hypothesis= c(0,1,-1,0)
linearHypothesis(franck_gpt2, hypothesis, 
                 vcov. =vcov(franck_gpt2)[["cond"]], 
                 coef. =fixef(franck_gpt2)[["cond"]])
